In [1]:
import numpy as np
import tensorflow as tf
import re
import time

In [2]:
#IMPORTING DATASET LINES AND CONVERSATIONS
lines=open("movie_lines.txt", encoding="utf-8", errors="ignore").read().split("\n")
lines[:4]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?']

In [3]:
movie=open("movie_conversations.txt", encoding="utf-8", errors="ignore").read().split("\n")
movie[:4]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']"]

In [4]:
#MAKING DICTIONARY LINEID WITH LINE NUMBER 
id2line={}
for line in lines:
    _line=line.split(" +++$+++")
    if len(_line)==5:
        id2line[_line[0]]=_line[4]
        
#id2line will have key, value pairs

In [5]:
#MAKING LIST OF CONVERSATIONS
conversation_list=[]
for conversations in movie:
    _conversation=conversations.split(" +++$+++ ")[-1][1:-1].replace("'", "").replace(" ","")
    conversation_list.append(_conversation.split(','))
    
conversation_list[:3]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203']]

In [6]:
#WE WILL MAKE SEPARATE LIST FOR QUESTIONS AS WELL AS FOR ANSWERS QITH CORRRESPOND TO EACH OTHER
questions=[]
answers=[]
for conversation in conversation_list:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

questions[:4]

[' Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 " Well, I thought we'd start with pronunciation, if that's okay with you.",
 ' Not the hacking and gagging and spitting part.  Please.',
 " You're asking me out.  That's so cute. What's your name again?"]

In [7]:
answers[:4]

[" Well, I thought we'd start with pronunciation, if that's okay with you.",
 ' Not the hacking and gagging and spitting part.  Please.',
 " Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?",
 ' Forget it.']

In [8]:
#DECLARIING FUNCTION FOR TEXT CLEANING
def clean_text(text):
    text=text.lower()
    text=re.sub(r"i'am", "i am", text)
    text=re.sub(r"he's","he is", text)
    text=re.sub(r"she's","she is", text)
    text=re.sub(r"that's","that is", text)
    text=re.sub(r"what's","ehat is", text)
    text=re.sub(r"\'ll", "will", text)
    text=re.sub(r"\'ve", "have", text)
    text=re.sub(r"\'re", "are", text)
    text=re.sub(r"\'d","would",text)
    text=re.sub(r"won't","will not", text)
    text=re.sub(r"can't","can not", text)
    text=re.sub(r"[-()\"#/@;:<>{}+=~|.?,]","" , text)
    return text

In [9]:
#APPLYING FUNCTION ON QUESTION AND ANSWERS
clean_questions=[]
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_questions[:4]

[' can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again',
 ' well i thought wewould start with pronunciation if that is okay with you',
 ' not the hacking and gagging and spitting part  please',
 ' youare asking me out  that is so cute ehat is your name again']

In [10]:
#APPLYING FUNCTION FOR ANSWERS LIST
clean_answers=[]
for answer in answers:
    clean_answers.append(clean_text(answer))
    
clean_answers[:4]

[' well i thought wewould start with pronunciation if that is okay with you',
 ' not the hacking and gagging and spitting part  please',
 " okay then how 'bout we try out some french cuisine  saturday  night",
 ' forget it']

In [11]:
#COUNT WORD FREQUENCIES OF WORDS FOR QUESTIONS AND ANSWERS

word2count={}
for i in clean_questions:
    for j in i.split():
        if j not in word2count:
            word2count[j]=1
        else:
            word2count[j]+=1
            
for i in clean_answers:
    for j in i.split():
        if j not in word2count:
            word2count[j]=1
        else:
            word2count[j]+=1
            
#word2count.items()

In [12]:
# WE WILL REMOVE WORDS WHICH ACCOURS LESS THAN 10 TIMES AND ADD IF IT OCUR MORE THAN 10 TIMES
#& ASSIGNED EACH WORD WHICH OCCUR MORE THAN 10 TIMES TO UNIQUE INTEGER

threshold=10
questionwords2int={}
word_number=0
for word, count in word2count.items():
    if count > threshold:
        questionwords2int[word]=word_number
        word_number+=1
        
answerwords2int={}
word_number=0
for word, count in word2count.items():
    if count > threshold:
        answerwords2int[word]=word_number
        word_number+=1
        
        


In [13]:
#WE WILL ADD 4 TOKENS WHICH ARE VERY ESSENTIAL FOR AUTOENCODERS(ENCODER AND DECODER)
#1.PAD
#2.SOS
#3.EOS
#4.OUT

#PAD->>FOR MAKING SAME LENTH FOR ENCODER AND DECODER CONTEXT VECTOR

#OUT->> FOR REPLACING ALL WORDS WHICH OCCUR LESS THAN 10 TIMES WITH ONE WORD "OUT" token

#SOS->>FOR START OF SENTENCE IN DECODING 

#EOS->> FOR END OF SENTENCE IN DECODING 

#FINALLY WE WILL THESE $ TOKENS IN BOTH LIST QUESTION AND ANSWERS AND WLL GIVE ALSO
# UNIQUE INTEGER NUMBER

tokens=['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    questionwords2int[token]=len(questionwords2int) + 1

for token in tokens:
    answerwords2int[token]=len(answerwords2int) +1 

In [14]:
#########################################"DOUBT"####################################

# WILL AMKE INVERSE ANSWERWORDS2INT DICTIONARY WHICH WILL BE USEFUL FOR LATER IN SEQ2SEQ MODEL
# here w_i and w are the keywords parameters

#1.w_i->>words_integer
#2.w->>words

answerint2words={w_i:w for w, w_i in answerwords2int.items()}

In [15]:
# ADD <EOS> TO THE END OF EACH SENTENCE IN ORDER TO GET BETTER UNDERSTANDING FOR DECODER

for i in range(len(clean_answers)):
    clean_answers[i]+=" <EOS>"

In [16]:
clean_answers[:3]

[' well i thought wewould start with pronunciation if that is okay with you <EOS>',
 ' not the hacking and gagging and spitting part  please <EOS>',
 " okay then how 'bout we try out some french cuisine  saturday  night <EOS>"]

In [17]:
# CONVERT EACH WORDS  OF QUESTION AND ANSERS INTO INTEGERS.
# REPLACE WORDS WHICH ARE LESS THAN THRESHOLD WITH TOKEN <"OUT">

question_into_integers=[]
for question in clean_questions:
    ints=[]
    for word in question.split():
        if word not in questionwords2int:
            ints.append(questionwords2int['<OUT>'])
        else:
            ints.append(questionwords2int[word])
    question_into_integers.append(ints)
    

answer_into_integers=[]
for answer in clean_answers:
    ints=[]
    for word in answer.split():
        if word not in answerwords2int:
            ints.append(answerwords2int['<OUT>'])
        else:
            ints.append(answerwords2int[word])
    answer_into_integers.append(ints)
    

In [18]:
answer_into_integers[:3]

[[18, 19, 20, 21, 22, 23, 13628, 24, 25, 26, 27, 23, 28, 13627],
 [29, 16, 30, 5, 13628, 5, 31, 32, 33, 13627],
 [27, 79, 104, 1680, 1, 944, 37, 527, 417, 2966, 224, 259, 13627]]

In [19]:
question_into_integers[:2]

[[0,
  1,
  2,
  3,
  4,
  13628,
  13628,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  13628,
  12,
  13,
  14,
  15,
  16,
  13628,
  17],
 [18, 19, 20, 21, 22, 23, 13628, 24, 25, 26, 27, 23, 28]]

In [20]:
len(question_into_integers[2])

9

In [21]:
question_into_integers[0:4]

[[0,
  1,
  2,
  3,
  4,
  13628,
  13628,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  13628,
  12,
  13,
  14,
  15,
  16,
  13628,
  17],
 [18, 19, 20, 21, 22, 23, 13628, 24, 25, 26, 27, 23, 28],
 [29, 16, 30, 5, 13628, 5, 31, 32, 33],
 [34, 35, 36, 37, 25, 26, 38, 39, 40, 26, 41, 42, 17]]

In [22]:
# I WILL SORT QUESTIONS AND ANSWERS BY THE SPECIFIC OPTIMUM LENGTHS FOR BETTER RESULTS AND LESS LOSS
# I WILL SET LENGTH = 35 IN THIS CASE

sorted_question_list=[]
sorted_answer_list=[]

for length in range(1, 35 + 1):
    for i in enumerate(question_into_integers):
        if len(i[1]) == length:
            sorted_question_list.append(question_into_integers[i[0]])
            sorted_answer_list.append(answer_into_integers[i[0]])

In [23]:
answer_into_integers[:2]

[[18, 19, 20, 21, 22, 23, 13628, 24, 25, 26, 27, 23, 28, 13627],
 [29, 16, 30, 5, 13628, 5, 31, 32, 33, 13627]]

In [24]:
sorted_question_list[:10]

[[52], [67], [130], [155], [142], [43], [186], [43], [193], [194]]

In [25]:
sorted_question_list[:3]

[[52], [67], [130]]

In [26]:
sorted_answer_list[:3]

[[16,
  53,
  26,
  52,
  54,
  55,
  16,
  56,
  57,
  49,
  58,
  59,
  60,
  57,
  61,
  45,
  62,
  19,
  0,
  29,
  63,
  64,
  65,
  66,
  13627],
 [68,
  69,
  65,
  70,
  71,
  72,
  73,
  74,
  75,
  65,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  65,
  84,
  85,
  57,
  80,
  86,
  87,
  13627],
 [109, 13627]]

# BUILDING SEQUENCE TO SEQUENCE MODEL

# TENSORFLOW
    

In [27]:
# WE WILL USE TENSORFLOW 

# WE WILL DEFINE INPUTS THARE NECCESSARY IN TNSORFLOW

# CREATE TENSOR PLACEHOLDERS of ALL BELOW VARIABLE ATLEAT
#1.INPUT
#2.TARGET VARIABLE("datatype", "shape of input data as well as target", "name for placeholder")
#3.LEARNING RATE
#4.KEEP_PROB_RATE THAT CONTROLS DROP OUT RATE IN TENSORFLOW

def model_inputs():
    inputs=tf.placeholder(tf.int32, [None, None], name="input")
    targets=tf.placeholder(tf.int32, [None, None], name="target")
    learning_rates=tf.placeholder(tf.float32, name="learning_rate")
    keep_prob=tf.placeholder(tf.float32, name="keep_prob")
    return inputs, targets, learning_rates, keep_prob

In [28]:
################AS DECODING LAYER EXPECT SOE FORMAT FOR TAKING INPUTS:->>>.>################
# SO WE WILL HAVE TO PREPROCESS DATA FIRST OF ALL
# WE WILL MAKE FUNCTION FOR THIS:
# WE WILL LEAVE LAST COLUMNS OF ANSWER("TARGET VARIABLE") AND WILL ADD "<SOS>" ON STARTING OF EACH 
# - ANSWER AND WLL LET THE DECODER TO GUESS THE LAST WORD
# WE NEED TWO THINGS FOR TAKING DATA INTO CORRECT FORMAT
#1.BATCH_SIZE->> DECODER LAYER WORKS IN BATCH SIZE NOT INDIVIDUALLY.
#2.IT REQUIRES <SOS> AT THE STARTING OF ANSWER("TARGET VARIABLE") SOE WE HAVE TO ADD IT IN FRONT
# OF EACH ROW OF BATCH
######## WE WILL AMKE FUNCTION PREPROCESSED TEXT--("PREPROCESSED_TARGET")--->>>>

# left=(matrix for <SOS>, word2int dictionary)
# right=(input, starting position, end position, stride of 1 by 1).


def preprocessed_targets(target, word2int, batch_size):
    left=tf.fill([batch_size, 1], word2int['<SOS>'])
    right=tf.strided_slice(target, [0, 0],[batch_size, -1], [1,1])
    preprocessed_target=tf.concat([left,right], 1)
    return preprocessed_target

In [13]:
# MAKE ENCODER LAYER AND WILL USE BIDIRECTIONAL RNN AND WILL USE VARIOUS LIBRARY OF TENSORFLOW
# AND PARAMETERS
# rnn_inputs=model inputs
# rnn_size=number of neurons
# keep_prob=it is used to control drop rates.
# sequence_length=list of length of questions of per batch_size.
#num_layers=number of layers.

def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
###### INITIALIZING SIMPLE RNN LAYER#################################################
    lstm=tf.contrib.rnn.BasicLSTMCell(rnn_size)
########### MAKING DROPOUT WITH ITS CONTROLLING PARAMETER############################
    lstm_dropout=tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=keep_prob)
######### MAKE ENCODER CELL ("INTERNAL MAKE MULTIRNNS")############################
    encoder_cell=tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
###### USING BIDIRECTIONAL RNN(FORWARD LAYER AND BACKWARD LAYER ALSO)################
    encoder_output, encoded_state=tf.nn.Bidirectional_dynamic_rnn(cell_for=encoder_cell,
                                                 cell_backward=encoder_cell,
                                                 sequence_lenth=sequence_length,
                                                 inputs=rnn_inputs,
                                                 dtype=tf.float32)
#### WE WILL USE ONE OF THIS OUTPUT AS "encoder-state".
    return encoder_state

In [ ]:
########################## ""DECODING THE TRAINING SET"" ############################################
### MAKE ATTENTION_KEYS, ATTENTION_STATE, ATTENTION_SCORE_FUNCTION, ATTENTION CONSTRUCT FUNCTION ####

#1.ATTENTION KEYS-> keys to be compared with target_state.
#2.ATTENTION VALUES-> USED TO CONSTRUCT CONTEXT VECTOR(RETURNED BY ENCODER AND WILL BE USED BY DECODER
#AS FIRST ELEMENT OF DECODING)

#3.ATTENTION_SCORE_FUNCTION->USED TO COMPUTE SIMILARITY B/W KEYS AND TARGET_STATE.
#4.ATTENTION_CONSTRUCT_FUINCTION-> USED TO BUILT ATTENTION STATE
### WE WILL USE ONE OF THREE OUTPUT WHICH IS DECODER_OUTPUT ##########################

In [19]:
def decoding_training_set(encoder_state,decoder_cell,decoder_embedded_input,sequence_length,decoding_scope,output_function,keep_prob,batch_size):
    attention_state=tf.zeros([batch_size,1,decoder_cell.output_size])
    attention_keys,attention_values,attention_score_function,attention_construct_function=tf.contrib.seq2seq.prepare_attention(
                                                                                                               attention_states,
                                                                                                               attention_options='bahdanau',
                                                                                                               num_units=decoder_cell.output_size)
    training_decoder_function=tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                           attention_keys,
                                                                           attention_values,
                                                                           attention_score_function,
                                                                           attention_construct_function,
                                                                           name='atte_dec_train')
    decoder_output,decoder_final_state,decoder_final_context=tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cel,
                                                                                                   training_decoder_function,
                                                                                                   sequence_lenth,
                                                                                                   decoder_embedded_input,
                                                                                                   scope=decoding_sope)
    decoder_output_dropout=tf.nn.dropout(decoder_output,keep_prob)
    return output_function(decoder_output_dropout)

In [20]:
########################## ""DECODING THE TRAINING SET"" ############################################
#def decoding_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length,output_function,batch_size, decoding_scope, keep_prob):
    #attention_states=tf.zeros([batch_size, 1, decoder_cell.output_size])
    
### MAKE ATTENTION_KEYS, ATTENTION_STATE, ATTENTION_SCORE_FUNCTION, ATTENTION CONSTRUCT FUNCTION ####

#1.ATTENTION KEYS-> keys to be compared with target_state.
#2.ATTENTION VALUES-> USED TO CONSTRUCT CONTEXT VECTOR(RETURNED BY ENCODER AND WILL BE USED BY DECODER
#AS FIRST ELEMENT OF DECODING)

#3.ATTENTION_SCORE_FUNCTION->USED TO COMPUTE SIMILARITY B/W KEYS AND TARGET_STATE.
#4.ATTENTION_CONSTRUCT_FUINCTION-> USED TO BUILT ATTENTION STATE
    #attention_keys, attention_values, attention_score_function, attention_construct_function=tf.contrib.seq2seq.prepare_attention(attention_states,
                                                                                                                                  attention_options='bahdanau',
                                                                                                                                  num_units=decoder_cell.output_size)
   # training_decoder_function=tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state=[0],
                                                                            attention_keys,
                                                                            attention_values,
                                                                            attention_score_function,
                                                                            attention_construct_function,
                                                                            name='atte_dec_train')
### WE WILL USE ONE OF THREE OUTPUT WHICH IS DECODER_OUTPUT ##########################
    #decoder_output, decoder_final_state, decoder_final_context=tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                     training_decoder_function,
                                                                                                     sequence_length,
                                                                                                     decoded_embedded_input,
                                                                                                     scope=decoding_scope)
                                                                                                     
    #decoder_output_dropout=tf.nn.dropout(decoder_output, keep_prob)
   # return output_function(decoder_otput_layer)

IndentationError: unexpected indent (<ipython-input-20-ff4de18bd3a2>, line 14)

In [1]:
############################################## DECDONG TEST SET #####################################
def decode_test_set(encoder_state,decoder_cell, decoder_embeddings_matrix, sos_id,eos_id, maximum_length, num_word,
                   sequence_length, decoding_scope, output_function,batch_size, keep_prob):
    attention_states=tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function,attention_construct_function=tf.contrib.seq2seq.prepare_attention(
                                                                  attention_states, attention_options='bahdanau', num_units=
                                                                                                       decoder_cell.output_size)
    
    test_decoder_function=tf.contrib.seq2seq.attention_decoder_fn_inference(output_function,
                                                                            encoder_state[0],
                                                                            attention_keys,
                                                                            attention_values,
                                                                            attention_score_function,
                                                                            attention_construct_function,
                                                                            decoder_embeddings_matrix,
                                                                            sos_id,
                                                                            eos_id,
                                                                            num_word,
                                                                            maximum_length,
                                                                            name='dec_atte_test')
                                                                           
    test_predictions,decoder_final_state,decoder_final_context=tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                   test_decoder_function,
                                                                                                   scope=decoding_scope)
    return test_predictions

In [ ]:
def decoder_rnn(decoder_embedded_input, decoder_embedded_matrix,encoder_state,num_words,sequence_length,rnn_size,num_layers,
                                                                                                                   word2int,
                                                                                                                   keep_prob,
                                                                                                                   batch_size):
    
 